### Let's modify the regularizer to boost the quality of training
#### we'll weight KL divergence term by a small weight (0.01) to boost training the decoder, and also we'll not compute the exact KL divergence term, instead we'll use monte carlo estimation

In [12]:
import tensorflow as tf
#from tensorflow.keras.models import Sequential, Model
import tf_keras as tfk
import tensorflow_probability as tfp
import random
import numpy as np
tfd = tfp.distributions
tfpl = tfp.layers
tfb = tfp.bijectors

import tf_keras as tfk
tfkl = tfk.layers

from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

In [13]:
DATA_PATH = "c:\\users\\talibit\\Drug_discovery_data\\alzheimersdata.txt"
data = open(DATA_PATH, 'r').read().splitlines()
new = []
count =0
for molecule in data:
    count+=1
    if 30 < len(molecule) < 50 and count < 300000:
        if ('T' not in molecule) and ('V' not in molecule) and ('g' not in molecule) and ('L' not in molecule) and ('8' not in molecule):
            new.append(molecule)

max_seq_len = len(max(new, key=len)) 

l = [['$']+ list(i) for i in new]
#l = [['$']+ [i] for i in new]
data = pad_sequences(l, maxlen=max_seq_len+1, padding="post", value="£",dtype=object) 
print(data.shape)
# so now each smile string is 50 character long element
#data = np.expand_dims(data, -1)
type(data)

(847, 50)


numpy.ndarray

In [14]:
tokenizer = Tokenizer(num_words=None, char_level=True, lower=False) 

tokenizer.fit_on_texts(data) 
new_data = tokenizer.texts_to_sequences(data) 
new_data = np.array(new_data)
new_data = np.expand_dims(new_data,-1)

In [15]:
word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index)

In [16]:
seed = 1
np.random.seed(seed)
full_train, x_test = train_test_split(np.array(new_data), test_size=0.2, random_state=seed)

x_train, x_val = train_test_split(np.array(full_train), test_size=0.1, random_state=seed)

In [30]:
latent_size = 10
Prior_distribution = tfd.MultivariateNormalDiag(loc=tf.zeros(latent_size, dtype=tf.float32),
                                        scale_diag=tf.ones(latent_size, dtype=tf.float32))

kl_regularizer1= tfpl.KLDivergenceRegularizer(Prior_distribution,
                                                          weight = 0.01,
                                                          use_exact_kl = False, 
                                                          test_points_fn = lambda q :q.sample(3),
                                                          test_points_reduce_axis = None) 

In [31]:
encoder1 = tfk.Sequential([
        tfkl.LSTM(128, input_shape=(50,1), return_sequences=True),
        tfkl.Dropout(0.15),
        tfkl.MaxPooling1D(pool_size=2),
        tfkl.LSTM(256, return_sequences=True),
        tfkl.Dropout(0.05),
        tfkl.MaxPooling1D(pool_size=2),
        tfkl.LSTM(512, return_sequences=True),
        tfkl.Dropout(0.25),
        tfkl.Flatten(),
        tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(latent_size)),
        tfpl.MultivariateNormalTriL(latent_size,
                                   activity_regularizer = kl_regularizer1)


    ])
encoder1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 50, 128)           66560     
                                                                 
 dropout_12 (Dropout)        (None, 50, 128)           0         
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 25, 128)           0         
 g1D)                                                            
                                                                 
 lstm_13 (LSTM)              (None, 25, 256)           394240    
                                                                 
 dropout_13 (Dropout)        (None, 25, 256)           0         
                                                                 
 max_pooling1d_5 (MaxPoolin  (None, 12, 256)           0         
 g1D)                                                 

In [32]:
decoder1 = tfk.Sequential([
    tfkl.Dense(10, activation = 'relu', input_shape =(latent_size,)),
    tfkl.Reshape((latent_size,1)),
    tfkl.LSTM(512, return_sequences=True),
    tfkl.Dropout(0.05),
    tfkl.UpSampling1D(size = 2),
    tfkl.LSTM(256, return_sequences=True),
    tfkl.Dropout(0.05),
    tfkl.UpSampling1D(size = 2),
    tfkl.LSTM(128, return_sequences=True),
    tfkl.Dropout(0.05),
    tfkl.Flatten(),
    tfkl.Dense(50 * (vocab_size+1)),  # Output 50 * 37 logits
    tfkl.Reshape((50, vocab_size+1)),  # Reshape to (50, 37)
    tfpl.DistributionLambda(lambda t: tfd.Independent(
        tfd.Categorical(logits=t), reinterpreted_batch_ndims=1))

])
decoder1.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 10)                110       
                                                                 
 reshape_4 (Reshape)         (None, 10, 1)             0         
                                                                 
 lstm_15 (LSTM)              (None, 10, 512)           1052672   
                                                                 
 dropout_15 (Dropout)        (None, 10, 512)           0         
                                                                 
 up_sampling1d_4 (UpSamplin  (None, 20, 512)           0         
 g1D)                                                            
                                                                 
 lstm_16 (LSTM)              (None, 20, 256)           787456    
                                                      

In [33]:
vae1 = tfk.Model(inputs=encoder1.inputs, outputs=decoder1(encoder1.outputs))

In [34]:
optimizer = tfk.optimizers.Adam(learning_rate=0.0005)
vae1.compile(optimizer=optimizer, loss=lambda x, pred : -tf.reduce_mean(pred.log_prob(x)))

In [35]:
vae1.fit(x = x_train, y = np.squeeze(x_train),validation_data = (x_val, np.squeeze(x_val)), epochs = 30)

Epoch 1/30
20/20 [==============================] - 13s 316ms/step - loss: 129.1586 - val_loss: 109.4745
Epoch 2/30
20/20 [==============================] - 5s 253ms/step - loss: 107.1041 - val_loss: 106.8401
Epoch 3/30
20/20 [==============================] - 5s 265ms/step - loss: 105.7166 - val_loss: 106.8946
Epoch 4/30
20/20 [==============================] - 5s 234ms/step - loss: 105.3808 - val_loss: 106.7333
Epoch 5/30
20/20 [==============================] - 5s 266ms/step - loss: 105.2827 - val_loss: 106.6230
Epoch 6/30
20/20 [==============================] - 5s 259ms/step - loss: 104.9072 - val_loss: 106.5135
Epoch 7/30
20/20 [==============================] - 5s 260ms/step - loss: 104.8665 - val_loss: 106.3804
Epoch 8/30
20/20 [==============================] - 5s 243ms/step - loss: 104.7489 - val_loss: 106.4742
Epoch 9/30
20/20 [==============================] - 5s 261ms/step - loss: 104.5439 - val_loss: 106.1882
Epoch 10/30
20/20 [==============================] - 5s 239ms/s

In [36]:
# Evaluate the model on the test set
test_loss = vae1.evaluate(x_test)
print("Test loss: {}".format(test_loss))

6/6 [==============================] - 2s 20ms/step - loss: 0.0456
Test loss: 0.04564611241221428


In [37]:
# Reconstruction 

random_number = random.randint(0, 846)
print("the initial smile string is:",''.join(data[random_number,:].tolist()))

new_data = tokenizer.texts_to_sequences(data[random_number,:]) 
new_data = np.expand_dims(new_data,0)
new_data.shape
q = encoder1(new_data)
print("the learned surrogate posterior is:", q.tensor_distribution)
p = decoder1(q.mean())
print("the learned p(x/z):", p.tensor_distribution)

sequence = p.sample().numpy()
tokenizer.sequences_to_texts(sequence)

the initial smile string is: $CCCC(=O)Nc1n[nH]c2ncc(-c3cccnc3)cc12£££££££££££££
the learned surrogate posterior is: tfp.distributions.MultivariateNormalTriL("MultivariateNormalTriL", batch_shape=[1], event_shape=[10], dtype=float32)
the learned p(x/z): tfp.distributions.Independent("IndependentCategorical", batch_shape=[1], event_shape=[50], dtype=int32)


['$ C c C c c C C C c [ C c H c 2 c ( c ( n N ) c 3 2 c 2 - - n c n 2 n 3 c 3 c c 2 £ 2 £ £ £ £ £ £ £']

In [38]:
# Generation
n_samples = 7000
samples = Prior_distribution.sample(n_samples)
output_distribution = decoder1(samples)
smiles = tokenizer.sequences_to_texts(output_distribution.sample().numpy())
smiles

['$ C l c 1 c n C ( c 2 c n C C c c C 1 = c 3 4 C c n c c c ) c 2 4 N ( ) N C c £ 3 £ £ C N £ £ £ £ £',
 '$ C O c n C c c c c c c c n C ( N c c ( 2 ) - c c c = ( 2 ( c F ) c ( O ( ) ) 2 C ) £ £ £ £ £ £ £ £',
 '$ C C c 1 c N c - = 1 1 ) C n C n 1 n ) c ( - ) O c C n = c c N l 4 C ) O ( O 2 £ 1 £ ) 1 ) £ £ £ £',
 '$ C O c C c c c c c 1 C 2 C 2 ï c c 2 c c n ( n c 3 C 2 c ) 3 c c c 2 ( c O n £ 1 £ C 1 £ £ £ c £ £',
 '$ C c c 1 c c c ( c c 2 c C n c - 2 - c ( 4 l c 3 c c c c ) 3 F c 2 = 1 C ) £ O £ £ £ 1 £ £ c £ £ £',
 '$ C O c c c C c c c 1 2 S C c c ( 2 ( = ( c ) n c ( c n c c c ) 3 c ) n C C £ ) N £ C N £ £ £ £ 1 £',
 '$ C c ( C ) c c 1 c 1 c c c 3 N ( C c c ) 4 N C ( ) C 2 c c N ) 2 n c N 2 1 2 2 C ) £ 1 1 £ £ C O £',
 '$ C O ( 1 c c c c 1 c C 1 n C c - 1 2 c c C - n = = c c c C 3 c C o = 2 [ ) O 2 c C C C £ £ £ £ £ £',
 '$ C O c n c c c 1 c 1 c 1 ( c c 3 n N n c ( - 2 = c c 2 ( 1 o ) ) c c £ 1 ) ) £ 1 £ £ £ £ £ £ £ £ £',
 '$ C O c n C c ( ( C 1 c N c n F ( ) n 2 c ( ) c 3 n N C c c c 

In [39]:
from rdkit import Chem
def checkSMILES(smiles):
	valid_smiles = []
	for smile in smiles:
		new = ''
		for char in smile:
			if char != '$' and char != '£':	
				new += char

		if Chem.MolFromSmiles(new, sanitize=False) is not None and len(new) > 15:
			valid_smiles.append(new)
			print(new)

	return valid_smiles

In [40]:
checkSMILES(smiles)

n C O c 1 c c c ) c c C 1 c 3 ( n 2 ( N c - 4 C N c c n n n n 2 4 ( 1 c  3 4  C  2   )  .  
S C c c 1 c C c c c 1 ) C C n N c c 3 - c ( N c c c c 3 1 3 n ) ) 3 c    O C   3 2  N    
P C l c 1 c 2 c c 3 1 C 1 c 3 C ( c C 3 ( C + C n c c C ) B 3 c c 2 ) ) ) ) = =  C  C 1 O c   
B F C c 1 + ( c c ( N N N ) C c c c O l - ( ( c O c c n c c c ) C c 3 = c = c   C 2 1  )  1  
n C O 1 1 c c c c c 1 2 1 1 C N ( C c = - - c 3 3 c c ( c 2 3 C ) ) 3  1 ) C ]   C  1  c 1  
c C C 1 1 c O c c ( ( 2 1 2 C ( C c ( ( c ( ( c 2 c c o c c C c o 4 4 ( C )  2 ] ) C » C = F   
N C c ( 1 c c c n ( 2 C C C C C ( c ( c C = N c ) c c ( N ( 3 F c ) ( ) = F C n   2 1    1  
S C c c n c c c c ( C [ - N 2 ( 2 1 ( c c c 4 = 3 c c - ( c 3 3  4 ( s  c ] ) N c C       
I F O c 1 c c c c ( c 2 1 C 1 N c c 2 c c ) c ( ) c c n 2 c n ) 4 4 ] = (  = O 1 ) C = n  s C  
B C O c 1 c c c c O 1 2 1 n n c 2 - c ( - ( n n 3 n - c ( c 3 ) 4 = ) ) ( O ] n 1 ) N  1   F  1
n C O 1 c 1 c c c c 1 n 1 C c n ( 3 ( ] N 4 - C 2 ( c c B n c ) 

[09:50:35] SMILES Parse Error: syntax error while parsing: C l c 1 c n C ( c 2 c n C C c c C 1 = c 3 4 C c n c c c ) c 2 4 N ( ) N C c  3   C N     
[09:50:35] SMILES Parse Error: Failed parsing SMILES ' C l c 1 c n C ( c 2 c n C C c c C 1 = c 3 4 C c n c c c ) c 2 4 N ( ) N C c  3   C N     ' for input: ' C l c 1 c n C ( c 2 c n C C c c C 1 = c 3 4 C c n c c c ) c 2 4 N ( ) N C c  3   C N     '
[09:50:35] SMILES Parse Error: syntax error while parsing: C O c n C c c c c c c c n C ( N c c ( 2 ) - c c c = ( 2 ( c F ) c ( O ( ) ) 2 C )        
[09:50:35] SMILES Parse Error: Failed parsing SMILES ' C O c n C c c c c c c c n C ( N c c ( 2 ) - c c c = ( 2 ( c F ) c ( O ( ) ) 2 C )        ' for input: ' C O c n C c c c c c c c n C ( N c c ( 2 ) - c c c = ( 2 ( c F ) c ( O ( ) ) 2 C )        '
[09:50:35] SMILES Parse Error: syntax error while parsing: C C c 1 c N c - = 1 1 ) C n C n 1 n ) c ( - ) O c C n = c c N l 4 C ) O ( O 2  1  ) 1 )    
[09:50:35] SMILES Parse Error: Failed parsing SMILE

['n C O c 1 c c c ) c c C 1 c 3 ( n 2 ( N c - 4 C N c c n n n n 2 4 ( 1 c  3 4  C  2   )  .  ',
 'S C c c 1 c C c c c 1 ) C C n N c c 3 - c ( N c c c c 3 1 3 n ) ) 3 c    O C   3 2  N    ',
 'P C l c 1 c 2 c c 3 1 C 1 c 3 C ( c C 3 ( C + C n c c C ) B 3 c c 2 ) ) ) ) = =  C  C 1 O c   ',
 'B F C c 1 + ( c c ( N N N ) C c c c O l - ( ( c O c c n c c c ) C c 3 = c = c   C 2 1  )  1  ',
 'n C O 1 1 c c c c c 1 2 1 1 C N ( C c = - - c 3 3 c c ( c 2 3 C ) ) 3  1 ) C ]   C  1  c 1  ',
 'c C C 1 1 c O c c ( ( 2 1 2 C ( C c ( ( c ( ( c 2 c c o c c C c o 4 4 ( C )  2 ] ) C » C = F   ',
 'N C c ( 1 c c c n ( 2 C C C C C ( c ( c C = N c ) c c ( N ( 3 F c ) ( ) = F C n   2 1    1  ',
 'S C c c n c c c c ( C [ - N 2 ( 2 1 ( c c c 4 = 3 c c - ( c 3 3  4 ( s  c ] ) N c C       ',
 'I F O c 1 c c c c ( c 2 1 C 1 N c c 2 c c ) c ( ) c c n 2 c n ) 4 4 ] = (  = O 1 ) C = n  s C  ',
 'B C O c 1 c c c c O 1 2 1 n n c 2 - c ( - ( n n 3 n - c ( c 3 ) 4 = ) ) ( O ] n 1 ) N  1   F  1',
 'n C O 1 c 1 c c c c 1 

#### Only 12 new molecules generated

#### The loss has decreased compared to the base model, because we used beta vae version with a small weight penalizing the KL divergence term, but the generation is still not satisfactory. the simplicity of the prior may be the reason behind

### Next we'll try to use a more sophisticated version of the prior